In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold, train_test_split


In [2]:
import random as random

Creating the dataframe

In [3]:
import glob
import os

In [4]:
cluster_df = pd.read_csv("/Users/aadidass-vattam/Documents/CorselloLab/Cluster/clusters200_with_viability.csv")

In [5]:
cluster_df = cluster_df[cluster_df['Less than 30% Viable'] >= 2]

In [6]:
cluster_df

,Unnamed: 0,pert_name,Cluster,MOA,Less than 20% Viable,Less than 30% Viable
0,2428,funapide,0,SODIUM CHANNEL BLOCKER,1.0,5.0
1,3237,lifitegrast,0,LYMPHOCYTE FUNCTION-ASSOCIATED ANTIGEN NEGATIV...,1.0,6.0
2,2320,fk-409,0,GUANYLYL CYCLASE ACTIVATOR,3.0,5.0
3,552,argireline,0,NEUROTRANSMITTER RELEASE INHIBITOR,2.0,5.0
4,5918,vatinoxan,0,ADRENERGIC RECEPTOR ANTAGONIST,0.0,2.0
...,...,...,...,...,...,...
1137,1204,cbl0137,199,HISTONE CHAPERONE INHIBITOR,888.0,890.0
1138,1567,cpi-169,199,HISTONE LYSINE METHYLTRANSFERASE INHIBITOR,19.0,50.0
1139,1528,convallatoxin,199,CARDIAC GLYCOSIDE,888.0,890.0
1140,5517,temefos,199,CHOLINESTERASE INHIBITOR,4.0,13.0


In [7]:
cluster_df[cluster_df['Cluster'] == 8]

,Unnamed: 0,pert_name,Cluster,MOA,Less than 20% Viable,Less than 30% Viable
37,5810,tucidinostat,8,HDAC INHIBITOR,44.0,124.0
38,1444,citarinostat,8,HDAC INHIBITOR,3.0,8.0
39,5050,sbha,8,HDAC INHIBITOR,3.0,9.0
40,5483,tcs-2210,8,NEURAL STEM CELL INDUCER,12.0,39.0
41,5461,tc-h-106,8,HDAC INHIBITOR,12.0,38.0
42,1610,cxd101,8,HDAC INHIBITOR,440.0,574.0
43,2635,gsk3117391,8,HDAC INHIBITOR,644.0,748.0
44,3269,lmk-235,8,HDAC INHIBITOR,88.0,225.0
45,5852,uf-010,8,HDAC INHIBITOR,7.0,15.0
46,3495,merck60,8,HDAC INHIBITOR,274.0,394.0


In [8]:
toy_ex = cluster_df[cluster_df['Cluster']==8]

In [9]:
toy_ex = toy_ex.drop_duplicates(subset=['pert_name','MOA'])

In [10]:
paths = []
for drug in list(toy_ex['pert_name']):
    for file in glob.glob(f'/Users/aadidass-vattam/Documents/CorselloLab/Cluster/responses/{drug}*.csv'):
        paths.append(file)

In [11]:
if len(paths) > 10:
    paths = random.sample(paths, 10)

In [12]:
features = pd.read_csv("/Users/aadidass-vattam/Documents/CorselloLab/Cluster/processed_data/x-all.csv")

In [13]:
features.head()
features = features.fillna(0)

In [14]:
features.head()

,ccle_name,GE_TSPAN6,GE_TNMD,GE_DPM1,GE_SCYL3,GE_C1orf112,GE_FGR,GE_CFH,GE_FUCA2,GE_GCLC,...,MET_C56:8 TAG,MET_C56:7 TAG,MET_C56:6 TAG,MET_C56:5 TAG,MET_C56:4 TAG,MET_C56:3 TAG,MET_C56:2 TAG,MET_C58:8 TAG,MET_C58:7 TAG,MET_C58:6 TAG
0,127399_SOFT_TISSUE,3.243364,0.000000,6.863567,2.063503,3.916477,0.070389,0.056584,5.426600,3.820690,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1321N1_CENTRAL_NERVOUS_SYSTEM,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,143B_BONE,4.719731,0.000000,7.360540,1.778209,3.786596,0.014355,0.713696,5.393348,4.638074,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,170MGBA_CENTRAL_NERVOUS_SYSTEM,4.155425,0.124328,6.681590,2.430285,2.799087,0.815575,3.559492,7.252855,4.300856,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,184A1_BREAST,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
data_frames = []
count = 0
for file_path in paths:
    df = pd.read_csv(file_path)
    df = df[['LFC.cb','ccle_name']]
    df = df.merge(features, on="ccle_name")
    df = df.drop(columns=['ccle_name'])
    df['ID'] = count
    data_frames.append(df)
    count += 1
    
    



In [16]:
df = pd.concat(data_frames)

In [17]:
corr = dict(df.corrwith(df['LFC.cb']))


/opt/anaconda3/envs/refract/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/refract/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [18]:
top_features = sorted(corr, key=lambda k: abs(corr[k]), reverse=True)[:502]

In [19]:
if 'ID' not in top_features:
    top_features = top_features[:501]

In [20]:
top_features.append('ID')

In [21]:
data = df[top_features]

In [22]:
data.head()

,LFC.cb,GE_S100A6,GE_LIN28B,GE_ITGA3,GE_LGALS3,GE_SPATS2L,GE_LIF,GE_SYTL2,GE_TRIM71,GE_NCEH1,...,GE_FCHO1,GE_ITGB1,GE_PLAAT4,GE_PTPN12,GE_TSPAN7,GE_SBK1,GE_MYBL1,GE_CSAG1,GE_SKOR1,ID
0,-1.275388,11.162580,2.100978,7.009885,8.004838,6.171727,4.375735,4.473787,0.056584,1.757023,...,3.531069,8.331454,1.622930,5.356144,2.750607,2.918386,2.104337,0.214125,0.028569,0
1,-0.717885,5.500483,0.000000,1.731183,2.266037,0.454176,0.526069,0.505891,0.000000,1.650765,...,3.422233,3.954196,0.411426,3.778209,0.042644,0.028569,0.400538,0.056584,0.545968,0
2,-0.439110,9.284176,4.488644,0.604071,1.269033,1.195348,4.018812,0.226509,1.521051,2.283922,...,3.452859,6.907492,1.176323,4.218006,2.090853,0.056584,1.411426,0.263034,0.545968,0
3,-1.954180,7.268846,5.350497,0.695994,1.695994,3.431623,2.944858,0.333424,1.782409,2.533563,...,3.193772,7.403182,1.459432,4.869378,3.116032,0.084064,1.459432,0.070389,0.201634,0
4,-2.352701,8.296503,0.000000,0.584963,3.725741,1.695994,0.201634,0.485427,0.056584,2.718088,...,3.917432,6.196332,1.650765,5.150560,0.124328,0.310340,1.879706,0.000000,0.201634,0


In [23]:
data.to_csv("toy_data_filtered.csv")

In [24]:
X = data.iloc[:,1:]

In [25]:
y=data.iloc[:,0]

In [26]:
y

0     -1.275388
1     -0.717885
2     -0.439110
3     -1.954180
4     -2.352701
         ...   
896   -0.718682
897   -1.078348
898   -1.932802
899    0.013111
900   -5.983090
Name: LFC.cb, Length: 8984, dtype: float64

In [27]:
X = (X-X.mean())/X.std()

In [28]:
X['ID'] = data['ID']

In [29]:
X.head()

,GE_S100A6,GE_LIN28B,GE_ITGA3,GE_LGALS3,GE_SPATS2L,GE_LIF,GE_SYTL2,GE_TRIM71,GE_NCEH1,GE_TM4SF1,...,GE_FCHO1,GE_ITGB1,GE_PLAAT4,GE_PTPN12,GE_TSPAN7,GE_SBK1,GE_MYBL1,GE_CSAG1,GE_SKOR1,ID
0,0.366488,0.979300,0.406031,0.702490,0.307160,0.465135,0.701713,-0.271155,-1.148424,1.050072,...,0.806268,0.258499,-0.598159,0.209721,0.585603,1.197076,-0.574523,-0.611186,-0.514383,0
1,-1.264284,-0.553618,-1.489424,-1.341092,-2.733046,-1.305679,-1.192501,-0.338397,-1.212502,-1.356434,...,0.739144,-2.353373,-1.112870,-1.209354,-0.752649,-0.862205,-1.807553,-0.676921,0.871099,0
2,-0.174522,2.721393,-1.894143,-1.696125,-2.338941,0.300954,-1.325874,1.469179,-0.830686,-0.880767,...,0.758032,-0.591168,-0.787902,-0.813834,0.259559,-0.842242,-1.075979,-0.590778,0.871099,0
3,-0.754968,3.350220,-1.861136,-1.544084,-1.149841,-0.193056,-1.274835,1.779769,-0.680143,-0.539810,...,0.598243,-0.295393,-0.667622,-0.228039,0.766194,-0.822660,-1.041237,-0.671161,-0.050954,0
4,-0.458987,-0.553618,-1.901005,-0.821293,-2.072731,-1.454916,-1.202271,-0.271155,-0.568868,-0.960492,...,1.044552,-1.015511,-0.586333,0.024834,-0.712282,-0.661416,-0.737087,-0.700531,-0.050954,0


Training the model

In [30]:
def add_noise(features, noise_level=0.01):
    noise = torch.randn_like(features) * noise_level
    return features + noise

def scale_features(features, scale_factor=0.1):
    scale = 1 + (torch.randn_like(features) * scale_factor)
    return features * scale

In [31]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.features = torch.tensor(X.values, dtype=torch.float32)
        self.targets = torch.tensor(y.values, dtype=torch.float32).unsqueeze(1)  # Adding extra dimension for targets
        
        self.features_noise = add_noise(self.features)
        self.features_scaled = scale_features(self.features)
        self.features_both = scale_features(add_noise(self.features))
        
        self.all_features = torch.cat([self.features, self.features_noise, self.features_scaled, self.features_both])
        self.all_targets = self.targets.repeat(4, 1)
    
    def __len__(self):
        return len(self.all_features)
    
    def __getitem__(self, idx):
        return self.all_features[idx], self.all_targets[idx]

In [32]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [33]:
class ImprovedNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(ImprovedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.bn1 = nn.BatchNorm1d(hidden_size1)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.bn2 = nn.BatchNorm1d(hidden_size2)
        self.relu2 = nn.ReLU()
        
        self.dropout = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        
        out = self.fc2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        
        out = self.dropout(out)
        out = self.fc3(out)
        return out



In [34]:
input_size = 501
hidden_size = 128
output_size = 1
num_epochs = 100
batch_size = 32
learning_rate = 0.001

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
model = ImprovedNN(input_size, 512,256, output_size)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
train_losses = []
val_losses = []
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for i, (features, targets) in enumerate(train_loader):
        outputs = model(features)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}')
    train_losses.append(train_loss)

        
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for features, targets in val_loader:
            outputs = model(features)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')

print("Training complete.")

/opt/anaconda3/envs/refract/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch [1/100], Train Loss: 2029.4399
Epoch [1/100], Validation Loss: 2.5283
Epoch [2/100], Train Loss: 1777.2849
Epoch [2/100], Validation Loss: 2.4847
Epoch [3/100], Train Loss: 1616.5028
Epoch [3/100], Validation Loss: 2.3039
Epoch [4/100], Train Loss: 1473.9116
Epoch [4/100], Validation Loss: 2.2138
Epoch [5/100], Train Loss: 1415.7000
Epoch [5/100], Validation Loss: 2.1481
Epoch [6/100], Train Loss: 1337.0980
Epoch [6/100], Validation Loss: 2.2202
Epoch [7/100], Train Loss: 1277.7161
Epoch [7/100], Validation Loss: 2.1861
Epoch [8/100], Train Loss: 1226.5889
Epoch [8/100], Validation Loss: 2.1516
Epoch [9/100], Train Loss: 1201.5018
Epoch [9/100], Validation Loss: 2.1458
Epoch [10/100], Train Loss: 1169.1893
Epoch [10/100], Validation Loss: 2.1408
Epoch [11/100], Train Loss: 1164.5233
Epoch [11/100], Validation Loss: 2.0710
Epoch [12/100], Train Loss: 1134.4556
Epoch [12/100], Validation Loss: 2.2485
Epoch [13/100], Train Loss: 1121.2345
Epoch [13/100], Validation Loss: 2.0792
Epoc

In [36]:
finetune_X = X.iloc[data[data['ID'] == 0].index]

In [37]:
finetune_y = y.iloc[data[data['ID'] == 0].index]

In [38]:
for name, param in model.named_parameters():
    if name not in ['fc3.weight', 'fc3.bias']:
        param.requires_grad = False

In [39]:
#Finetune
new_epochs=100
X_train, X_test, y_train, y_test = train_test_split(finetune_X,finetune_y,test_size=0.33,random_state=42)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
for epoch in range(new_epochs):
    model.train()
    train_loss = 0
    for i, (features, targets) in enumerate(train_loader):
        outputs = model(features)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
        
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for features, targets in val_loader:
            outputs = model(features)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f'Epoch [{epoch+1}/{new_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

print("Training complete.")

Epoch [1/100], Train Loss: 183.4544, Validation Loss: 1.2876
Epoch [2/100], Train Loss: 73.8927, Validation Loss: 0.8941
Epoch [3/100], Train Loss: 61.0796, Validation Loss: 0.8175
Epoch [4/100], Train Loss: 54.8768, Validation Loss: 0.7309
Epoch [5/100], Train Loss: 49.8744, Validation Loss: 0.6839
Epoch [6/100], Train Loss: 47.7828, Validation Loss: 0.6363
Epoch [7/100], Train Loss: 44.9456, Validation Loss: 0.5979
Epoch [8/100], Train Loss: 44.0380, Validation Loss: 0.5806
Epoch [9/100], Train Loss: 43.2861, Validation Loss: 0.5768
Epoch [10/100], Train Loss: 42.2473, Validation Loss: 0.5523
Epoch [11/100], Train Loss: 41.8552, Validation Loss: 0.5574
Epoch [12/100], Train Loss: 41.7423, Validation Loss: 0.5359
Epoch [13/100], Train Loss: 41.9519, Validation Loss: 0.5319
Epoch [14/100], Train Loss: 40.8145, Validation Loss: 0.5274
Epoch [15/100], Train Loss: 39.6120, Validation Loss: 0.5300
Epoch [16/100], Train Loss: 40.0484, Validation Loss: 0.5306
Epoch [17/100], Train Loss: 40.5

In [40]:
#Train from scratch
new_epochs=200
X_train, X_test, y_train, y_test = train_test_split(finetune_X,finetune_y,test_size=0.33,random_state=42)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
model2 = ImprovedNN(input_size, 512,256, output_size)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
for epoch in range(new_epochs):
    model2.train()
    train_loss = 0
    for i, (features, targets) in enumerate(train_loader):
        outputs = model2(features)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
        
    model2.eval()
    with torch.no_grad():
        val_loss = 0
        for features, targets in val_loader:
            outputs = model2(features)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f'Epoch [{epoch+1}/{new_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

print("Training complete.")

Epoch [1/200], Train Loss: 111.5858, Validation Loss: 1.1007
Epoch [2/200], Train Loss: 107.7362, Validation Loss: 1.1024
Epoch [3/200], Train Loss: 109.1536, Validation Loss: 1.1146
Epoch [4/200], Train Loss: 109.2053, Validation Loss: 1.1130
Epoch [5/200], Train Loss: 110.6679, Validation Loss: 1.1169
Epoch [6/200], Train Loss: 109.0765, Validation Loss: 1.1101
Epoch [7/200], Train Loss: 109.7284, Validation Loss: 1.1063
Epoch [8/200], Train Loss: 106.9332, Validation Loss: 1.1079
Epoch [9/200], Train Loss: 110.2005, Validation Loss: 1.1067
Epoch [10/200], Train Loss: 106.7441, Validation Loss: 1.1096
Epoch [11/200], Train Loss: 110.4850, Validation Loss: 1.1169
Epoch [12/200], Train Loss: 110.6533, Validation Loss: 1.1076
Epoch [13/200], Train Loss: 108.4519, Validation Loss: 1.1293
Epoch [14/200], Train Loss: 108.8656, Validation Loss: 1.1018
Epoch [15/200], Train Loss: 109.2217, Validation Loss: 1.1092
Epoch [16/200], Train Loss: 110.4191, Validation Loss: 1.1240
Epoch [17/200], T